In [ ]:
def to_hieararchical_form(features):
    if lang == "pol":
        features = features.split(";")
        word_class = features[0]
        if (word_class=="V"):
            for i in range (len(features)):                
                if features[i].isnumeric() or features[i]=="SG" or features[i]=="PL":
                    # SG;1 -> 1;SG
                    if features[i+1].isnumeric():
                        tmp = features[i+1]
                        features[i+1] = features[i]
                        features[i] = tmp
                    # V;...;1;SG -> ...;NOM(1,SG)
                    hierarchical = "NOM("+",".join(features[i:])+")"
                    result = ";".join(features[:i] + [hierarchical])
                    print(result)
                    return result
        elif (word_class=="N" or word_class=="ADJ"):
            for i in range (len(features)):
                if features[i]=="SG" or features[i]=="PL":
                    hierarchical = "("+",".join(features[i:])+")"
                    result = ";".join(features[:i]) + hierarchical
                    print(result)
                    return result
        else:
            exit("invalid word class: " + word_class)
    elif lang == "ote":
        if features[0]=="V":
            for i in range (len(features)):
                if features[i]=="SG" or features[i].isnumeric():
                    hierarchical = "NOM("+",".join(features[i:])+")"
                    return ";".join(features[:i] + [hierarchical])
        else: exit("invalid word class: ",word_class)
    else: exit("invalid language: " + lang)


def preprocess(x):
    x = x.strip()
    lemma, target, features = x.split("\t")
    processed_features = to_hierarchical_form(features)
    line = lemma + "\t" + processed_features + "\t" + target
    return line

path_data = "../data/"
LANGS = ["ote","pol"]
for lang in LANGS:
    with open (path + lang,"r") as data_file, open (path + lang + ".args","w") as output_file:
        data_lines = data_file.readlines()
        output_file.write("\n".join(map(preprocess,data_lines)))

In [ ]:
# split and sample from the paradigms according to the SIGMORPHON–UniMorph 2023 Shared Task 0 Paper
def get_lemma(line):
    lemma = line.split("\t")[0]
    return lemma

from random import sample,shuffle
from math import floor
from itertools import groupby
path_data = "../data/"
LANGS = ["ote","pol"]

for lang in LANGS:
    with open (path_data + lang,"r") as data_file, open (path_data + lang + ".trn","w") as train_file, open (path_data + lang + ".dev","w") as dev_file,open (path_data + lang + ".tst","w") as test_file:
        data_lines = [line.strip() for line in data_file.readlines()]
        tables = [(k, list(g)) for k, g in groupby(data_lines, get_lemma)]   
        
        sampled_tables = sample(tables,500)
        size = len(sampled_tables)
        datasets = [ sampled_tables[:floor(0.8*size)], sampled_tables[floor(0.8*size):floor(0.9*size)], sampled_tables[floor(0.9*size):] ]

        train_forms = [form for lemma,table in datasets[0] for form in table]
        dev_forms = [form for lemma,table in datasets[1] for form in table]
        test_forms = [form for lemma,table in datasets[2] for form in table]
        
        if len(dev_forms) < 1000 or len(test_forms) < 1000 or len(train_forms) < 10000:
            exit("more tables needed for lang: " + lang)
            
        # sampling and retaining order of the forms
        indices_train = sorted(sample(range(len(train_forms)), 10000))
        indices_dev =  sorted(sample(range(len(dev_forms)), 1000))
        indices_test = sorted( sample(range(len(test_forms)), 1000))

        train_samples = [train_forms[i] for i in indices_train]
        dev_samples = [dev_forms[i] for i in indices_dev]
        test_samples = [test_forms[i] for i in indices_test]

        train_txt = "\n".join(train_samples) 
        dev_txt = "\n".join(dev_samples) 
        test_txt = "\n".join(test_samples) 

        train_file.write(train_txt)
        dev_file.write(dev_txt)
        test_file.write(test_txt)
        
